In [232]:
import os
import sys
import numpy as np
import pandas as pd
import shutil

import torch
from torchvision import transforms
import cv2

import albumentations as A

from datetime import datetime

In [255]:
## from tensor to numpy function
fn_tonumpy = lambda x: x.to('cpu').detach().numpy()

## 리스트 펼치기 함수
def flatten(lst):
    result = []
    for item in lst:
        result.extend(item)
    return result

## 제출파일 저장하기
def save_submission(data_dir, result_dir, pred, epoch, batch):
    submission = pd.DataFrame()
    df = pd.read_csv('./data/val_images_relabeled/fold4.csv')
    submission['image_id'] =  df['image_id']
    submission['label'] = pred
    submission.to_csv('./data/val_images_relabeled/result/submission_f4.csv', index=False)


## 네트워크 불러오기
def load(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir):
        epoch = 0
        return net, optim, epoch

    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    #epoch.load_state_dict(dict_model['epoch'])
    #epoch = int(ckpt_lst[-1].split('epoch')[1].split('_batch')[0])

    return net, optim#, epoch

In [256]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, img_x, img_y, transform=None):
        self.df = df
        self.data_dir = data_dir
        self.img_x = img_x
        self.img_y = img_y
        self.transform = transform

        lst_label = list(df['label'])
        lst_input = list(x for x in df.image_id.values)
        self.lst_label = lst_label
        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        label = self.lst_label[index]
        input = cv2.imread(os.path.join(self.data_dir, self.lst_input[index]), cv2.IMREAD_COLOR)
        input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)  # result of input shape is y,x,c

        if self.transform:
            input = self.transform(image=input)['image']

        data = {'input': input, 'label' : label}

        return data

In [257]:
import timm

import torch
import torch.nn as nn

##
class CassvaImgClassifier(nn.Module):
    def __init__(self, network, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(network, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)

        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''

    def forward(self, x):
        x = self.model(x)
        return x

In [258]:
import os
import sys
import numpy as np
import pandas as pd

# pytorch
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp.autocast_mode import autocast
from torch.cuda.amp import GradScaler

# sci-kit learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2

def transform_test(img_x, img_y):
    return A.Compose([
            A.RandomResizedCrop(img_x, img_y),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
#             A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)], p=1.)

def dataloader_test(df, data_dir, img_x, img_y, batch_size):
    dataset_test = TestDataset(df=df, data_dir=data_dir, img_x=img_x, img_y=img_y, transform=transform_test(img_x, img_y))
    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=8)

    num_data_test = len(dataset_test)
    num_batch_test = np.ceil(num_data_test / batch_size)

    return loader_test, num_batch_test

In [259]:
def prob_to_digit(output, batch_size):
    np_output = fn_tonumpy(output)
    lst_output = []

    for i in range(batch_size):
        nb_output = np.argmax(np_output[i, :])
        lst_output.append(nb_output)

    return lst_output

In [260]:
def test(args):
    # hyperparameters
    img_x = args.img_x
    img_y = args.img_y

    lr = args.lr
    batch_size = args.batch_size

    num_epoch = args.num_epoch

    data_dir = args.data_dir
    ckpt_dir = args.ckpt_dir
    result_dir = args.result_dir

    network = args.network

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##
    print("Test start ... ")
    df = pd.read_csv('./data/val_images_relabeled/fold4.csv')

    loader_test, num_batch_test = dataloader_test(df=df, data_dir=data_dir, img_x=img_x, img_y=img_y, batch_size=batch_size)
    net = CassvaImgClassifier(network, df.label.nunique(), pretrained=False).to(device)
    optim = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-6)
    net, optim = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

    #used_epoch = [6, 7, 8, 9]  if test use rnd_epochs, it can have weights for each iteration.

    with torch.no_grad():
        net.eval()
        st_iter = 0
        tta = 1
        lst_pred = []

        acc_epoch_test = 0

        for batch, data in enumerate(loader_test, 1):
            # forward pass
            label = data['label'].to(device).long()
            input = data['input'].to(device).float()

            output = net(input)

            pred = torch.argmax(output, dim=1).cpu().detach().numpy()
            lst_pred.append(pred)  
            
            label = label.cpu().detach().numpy()
            acc_batch_test = accuracy_score(label, pred)
            acc_epoch_test += acc_batch_test

        
        test_acc = (acc_epoch_test / num_batch_test)
        prediction = flatten(lst_pred)
        
        print(test_acc)
#         print(prediction)


        # submission
        save_submission(data_dir=data_dir, result_dir=result_dir, pred=prediction, epoch=num_epoch, batch=batch_size)

In [261]:
import os
import numpy as np
import random

import torch

import argparse



## Parser 생성하기
parser = argparse.ArgumentParser(description="Cassava Leaf Disease Classification",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--mode", default="test", choices=["train", "test"], type=str, dest="mode")
parser.add_argument("--train_continue", default="off", choices=["on", "off"], type=str, dest="train_continue")

parser.add_argument("--seed", default=719, type=int, dest="seed")
parser.add_argument("--img_x", default=512, type=int, dest="img_x")
parser.add_argument("--img_y", default=512, type=int, dest="img_y")

parser.add_argument("--lr", default=1e-4, type=float, dest="lr")
parser.add_argument("--num_fold", default=10, type=int, dest="num_fold")
parser.add_argument("--num_epoch", default=10, type=int, dest="num_epoch")
parser.add_argument("--batch_size", default=4, type=int, dest="batch_size")

parser.add_argument("--cutmix", default=False, choices=[True, False], type=bool, dest="cutmix")
parser.add_argument("--fmix", default=False, choices=[True, False], type=bool, dest="fmix")

parser.add_argument("--label_smooth", default=False, choices=[True, False], type=bool, dest="label_smooth")
parser.add_argument("--swa", default=False, choices=[True, False], type=bool, dest="swa")

parser.add_argument("--data_dir", default="./data/val_images_relabeled/fold4", type=str, dest="data_dir")
parser.add_argument("--ckpt_dir", default="./data/val_images_relabeled/checkpoint", type=str, dest="ckpt_dir")
parser.add_argument("--result_dir", default="./data/val_images_relabeled/result", type=str, dest="result_dir")

parser.add_argument("--network", default="tf_efficientnet_b4_ns", type=str, dest="network")
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args, unknown = parser.parse_known_args()

##
if __name__ == "__main__":
    # random seed
    def seed_everything(seed):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

    seed = args.seed
    seed_everything(seed)

    if args.mode == "train":
        train(args)
    elif args.mode == "test":
        test(args)

Test start ... 
0.8759345794392523


In [ ]:
0.8764018691588785

In [ ]:
0.8953271028037383

In [ ]:
0.8968847352024921

In [ ]:
0.8994548286604361